<img style="float: left;" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Chat

In this module, the Semantic Kernel ability to create a chat experience will be introduced. This is a much richer experience than just sending simple prompts that are stateless and context is forgotten.

Semantic Kernel has first-class support for chat scenarios, where the user talks back and forth with the LLM, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. This allows the LLM to know the historical context of the conversation.

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [6]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// Retrieve the configuration settings for the Azure OpenAI service
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0

In [7]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.11.1"

using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName,
        endpoint: azureOpenAIEndpoint,
        apiKey: azureOpenAIAPIKey)
    .Build();

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();


Installed Packages Microsoft.Extensions.DependencyInjection, 8.0.0 Microsoft.SemanticKernel, 1.11.1

### Step 2 - Execute the Chat Completion Service with Prompt Execution Settings

Using the Semantic Kernel ChatCompletion service is very similar to inovoking a prompt for basic LLM interactions. The chat completion service will provide very similar results to invoking the prompt directly.

In [8]:
// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 2500, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var simpleDecisionPrompt = "Provide some decision frameworks that can help improve the quality of decisions.";
var chatResult = await chatCompletionService.GetChatMessageContentAsync(simpleDecisionPrompt, openAIPromptExecutionSettings);

Console.WriteLine(chatResult.Content);

1. SWOT Analysis: This framework helps identify the strengths, weaknesses, opportunities, and threats related to a decision. It provides a comprehensive view of the internal and external factors that can impact the decision.

2. Decision Matrix: This framework helps evaluate and prioritize different options based on specific criteria. Each option is scored based on how well it meets each criterion, and the option with the highest score is selected.

3. Cost-Benefit Analysis: This framework helps assess the costs and benefits of different options. It involves quantifying the costs and benefits in monetary terms and comparing them to determine which option provides the greatest net benefit.

4. Pareto Analysis: This framework helps identify the most significant factors that contribute to a problem or decision. It involves ranking factors based on their impact and focusing on the top factors that account for the majority of the impact.

5. Decision Tree: This framework helps visualize the

### Step 3 - Execute a Semantic Kernel Chat Completion with Streaming

Semantic Kernel supports prompt response streaming in the ChatCompletionService. This allows responses to be streamed to the client as soon as they are made available by the LLM and service. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

In [9]:
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(simpleDecisionPrompt))
{
    Console.Write(content);
}

1. SWOT Analysis: Analyzing strengths, weaknesses, opportunities, and threats helps to evaluate options and identify potential risks and benefits of various decisions.

2. Decision Matrix: This involves listing decision criteria and options, assigning weights to each criterion based on importance, and scoring each option against the criteria to compute a total score for each option, helping to assess which option is most favorable.

3. Cost-Benefit Analysis: Comparing the costs and benefits of alternatives allows for a quantifiable assessment of the potential outcomes of a decision.

4. Decision Trees: Mapping out different possible outcomes and paths in a tree-like structure, with branching decisions, allows individuals to foresee consequences and make informed choices based on probabilities.

5. Pareto Analysis: Utilizing the 80/20 rule, this identifies the most significant factors influencing a decision by focusing on the few causes that have the largest impact.

6. Multi-Criteria D

### Step 4 - Execute a Semantic Kernel with Chat History

Semantic Kernel has a ChatHistory object that can be used with the ChatCompletionService to provide historical chat context to the LLM. Notice that the ChatHistory object differentiates between the different types of chat messages:
* System Message - System or MetaPrompt
* User Message - A message from the user
* Assistante Message - A message from the LLM

This is a more sophisticated approach than passing chat history in a long dynamic string. ChatHistory objects can be serialized and persisted into databases as well.

In [10]:
var systemPrompt = """
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";
var simpleDecisionPrompt = "Provide five decision frameworks that can help improve the quality of decisions.";

var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(simpleDecisionPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

// private void WriteMessages(IReadOnlyList<ChatMessageContent> messages, ChatHistory? history = null)
// {
//     foreach (var message in messages)
//     {
//         Console.WriteLine($"# {message.Role}: {message.Content}");
//     }
//     history?.AddRange(messages);
// }
// WriteMessages(await chatCompletionService.GetChatMessageContentsAsync(chatHistory, openAIPromptExecutionSettings), chatHistory);

Certainly! Here are five decision frameworks that can help in improving decision-making quality across various contexts:

1. **Cost-Benefit Analysis (CBA)**:
   - **Description**: This framework involves weighing the costs and benefits of various options to determine which yields the highest net benefits. Costs and benefits are quantified in monetary terms or scored based on importance and impact.
   - **Ideal Use Case**: Best suited for financial decisions, investment opportunities, or evaluating resource allocation in project management.
   - **Steps**:
     1. List all option costs and benefits.
     2. Quantify costs and benefits in comparable terms.
     3. Compare net benefits (benefits minus costs) of each option.
     4. Select the option with the highest net benefits.

2. **SWOT Analysis**:
   - **Description**: This involves identifying Strengths, Weaknesses, Opportunities, and Threats related to a decision or project. It helps in understanding both internal and external fact

### Step 5 - Using Semantic Kernel Chat History in Conversation

With Semantic Kernel maintaining state with the ChatHistory object, a user can interact with the LLM in a natural way. Note the interaction below, where the user is asking which of the particular decision frameworks are best suited to purchase a car. Without the ChatHistory being passed along, the LLM would not have the proper context to answer the question.

In [11]:
var simpleDecisionPromptFollowup = "Which of these 5 decision frameworks would work best for making a decision to purchase a car. Select only the single best one.";

chatHistory.AddUserMessage(simpleDecisionPromptFollowup);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

For making a decision to purchase a car, the **Decision Matrix (or Grid Analysis)** would be the most effective framework among the ones listed. Here’s why it stands out as the best choice for this specific scenario:

### Rationale for Choosing Decision Matrix for Car Purchase:
1. **Multiple Factors**: Buying a car involves considering multiple factors such as price, brand, fuel efficiency, reliability, resale value, features, and aesthetic preferences. A decision matrix allows for all these factors to be listed and quantified.
 
2. **Customizable Criteria**: Each buyer may have different priorities or constraints (e.g., budget limitations, need for safety features, concern about environmental impact). The decision matrix framework facilitates the customization of criteria weights based on individual preferences, ensuring that the decision aligns with personal priorities.

3. **Quantitative Comparison**: Car buying decisions can benefit from a structured comparison, where each aspect i

In [12]:
var simpleDecisionPromptFollowupPartTwo = "Which of these 5 decision frameworks listed above are used by the military intelligence community?";

chatHistory.AddUserMessage(simpleDecisionPromptFollowupPartTwo);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

The military intelligence community often integrates various decision-making frameworks to address complex and dynamic security environments. Two frameworks from the list provided that are notably utilized within the military context are the **SWOT Analysis** and **Cost-Benefit Analysis (CBA)**. Here’s how each framework is utilized:

1. **SWOT Analysis**:
   - **Utility in Military Intelligence**: SWOT Analysis is used extensively in strategic planning within the military to assess the strengths, weaknesses, opportunities, and threats related to specific operations, potential military campaigns, or overall defense strategies. It aids in recognizing internal capabilities and external circumstances that could impact the success of a mission or the security posture of a military unit or nation.
   - **Application Example**: Before deploying troops or initiating a new operational strategy, military strategists might conduct a SWOT analysis to evaluate the preparedness of their forces (str

### Step 6 - Semantic Kernel inspecting Chat History 

The Semantic Kernel Chat History is a transparent object that can be inspected and written out. In fact, the ChatHistory object can be manipulated to replay chats from middle interactions to simulate different outcomes.

Execute the cell below to write out the Semantic Kernel Chat History of this chat.

In [13]:
Console.WriteLine("Number of chat interactions: " + chatHistory.Count());

foreach (var message in chatHistory)
{
    Console.WriteLine($"# {message.Role}: {message.Content}");
}

Number of chat interactions: 7
# system: You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
# user: Provide five decision frameworks that can help improve the quality of decisions.
# assistant: Certainly! Here are five decision frameworks that can help in improving decision-making quality across various contexts:

1. **Cost-Benefit Analysis (CBA)**:
   - **Description**: This framework involves weighing the costs and benefits of various options to determine which yields the highest net benefits. Costs and benefits are quantified in monetary terms or scored based on importance and impact.
   - **Ideal Use Case**: Best suited for financial decisions, investment opportunities, or evaluating resource allocation in project management.
   - **Steps**:
     1. List all option costs and benefits.
     2. Quantif